In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp /content/drive/MyDrive/AIExam/data.zip /content
!mkdir dataset
!unzip data.zip

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import csv
from PIL import Image
import numpy as np

# Function to load data from CSV and images
def load_data(file_path):
    data = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader, None)
        for row in csv_reader:
            text, image_path, emotion = row
            img = Image.open(image_path).convert('RGB')
            img = img.resize((64, 64))
            img_array = np.array(img) / 255.0
            entry = {'text': text, 'image_data': img_array, 'emotion': emotion}
            data.append(entry)
    return data

# Load data
file_path = "data/processed/text_image_emotion.csv"
data = load_data(file_path)

In [4]:
# Convert data to numpy arrays
texts = [entry['text'] for entry in data]
images = np.array([entry['image_data'] for entry in data])
emotions = [entry['emotion'] for entry in data]

# Encode emotions
label_encoder = LabelEncoder()
encoded_emotions = label_encoder.fit_transform(emotions)
one_hot_emotions = tf.keras.utils.to_categorical(encoded_emotions, num_classes=len(label_encoder.classes_))

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
text_sequences = tokenizer.texts_to_sequences(texts)
text_padded = pad_sequences(text_sequences, padding='post')

# Split data into training and testing sets
text_train, text_test, image_train, image_test, emotion_train, emotion_test = train_test_split(
    text_padded, images, one_hot_emotions, test_size=0.2, random_state=42
)

In [5]:
# Build the neural network model
text_input = layers.Input(shape=(text_padded.shape[1],), dtype=tf.int32)
embedding_layer = layers.Embedding(input_dim=max_words, output_dim=16, input_length=text_padded.shape[1])(text_input)
lstm_layer = LSTM(16)(embedding_layer)
text_flatten = layers.Flatten()(lstm_layer)

image_input = layers.Input(shape=(64, 64, 3))
conv_layer = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
pooling_layer = layers.MaxPooling2D((2, 2))(conv_layer)
flatten_layer = layers.Flatten()(pooling_layer)

# Concatenate the flattened text and image layers
concatenated = layers.Concatenate()([text_flatten, flatten_layer])
# Add Dropout for regularization
dropout_layer = layers.Dropout(0.5)(concatenated)
dense_layer = layers.Dense(64, activation='relu')(dropout_layer)
output_layer = layers.Dense(len(label_encoder.classes_), activation='softmax')(dense_layer)

model = models.Model(inputs=[text_input, image_input], outputs=output_layer)

# Compile the model with a lower learning rate
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Add ModelCheckpoint callback to save the best model
checkpoint_callback = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_accuracy", mode="max")

# Create an ImageDataGenerator for data augmentation
image_data_generator = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Aggiungi callback per l'early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 34)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 embedding (Embedding)       (None, 34, 16)               160000    ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 62, 62, 32)           896       ['input_2[0][0]']             
                                                                                              

In [6]:

model.fit([text_train, image_train], emotion_train, epochs=30, validation_data=([text_test, image_test], emotion_test), callbacks=[checkpoint_callback, early_stopping_callback])


Epoch 1/30
585/585 [==============================] - 28s 37ms/step - loss: 1.6526 - accuracy: 0.3435 - val_loss: 1.5725 - val_accuracy: 0.4030
Epoch 2/30
  9/585 [..............................] - ETA: 3s - loss: 1.5783 - accuracy: 0.4028

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


585/585 [==============================] - 6s 11ms/step - loss: 1.5129 - accuracy: 0.4290 - val_loss: 1.4618 - val_accuracy: 0.4600
Epoch 3/30
585/585 [==============================] - 7s 11ms/step - loss: 1.4362 - accuracy: 0.4655 - val_loss: 1.4302 - val_accuracy: 0.4662
Epoch 4/30
585/585 [==============================] - 8s 13ms/step - loss: 1.3786 - accuracy: 0.4879 - val_loss: 1.3383 - val_accuracy: 0.5152
Epoch 5/30
585/585 [==============================] - 7s 11ms/step - loss: 1.2416 - accuracy: 0.5446 - val_loss: 1.2641 - val_accuracy: 0.5363
Epoch 6/30
585/585 [==============================] - 5s 8ms/step - loss: 1.1609 - accuracy: 0.5813 - val_loss: 1.2334 - val_accuracy: 0.5498
Epoch 7/30
585/585 [==============================] - 4s 7ms/step - loss: 1.1039 - accuracy: 0.5985 - val_loss: 1.2356 - val_accuracy: 0.5479
Epoch 8/30
585/585 [==============================] - 5s 9ms/step - loss: 1.0643 - accuracy: 0.6135 - val_loss: 1.2163 - val_accuracy: 0.5564
Epoch 9/30
58

In [9]:
# Evaluate the model on the test data
evaluation_results = model.evaluate([text_test, image_test], emotion_test)
# Print the evaluation results
print(f"Loss: {evaluation_results[0]}, Accuracy: {evaluation_results[1]}")
# Make predictions on new test data
predictions = model.predict([text_test, image_test])
# Print the predicted probabilities for each class
print(predictions)

147/147 [==============================] - 1s 4ms/step - loss: 1.2105 - accuracy: 0.5609
Loss: 1.2104779481887817, Accuracy: 0.5608974099159241
147/147 [==============================] - 0s 3ms/step
[[8.81730055e-04 3.92039627e-04 3.64822889e-04 ... 4.17445414e-03
  7.53876986e-04 4.63532197e-04]
 [5.24454452e-02 2.31959708e-02 1.16633996e-02 ... 6.51205361e-01
  7.07607344e-02 5.63853085e-02]
 [1.79932699e-01 3.86128947e-02 1.06525704e-01 ... 2.71224659e-02
  3.36882547e-02 6.05008602e-01]
 ...
 [3.84986222e-01 1.61189493e-02 2.21460387e-02 ... 2.73633450e-01
  9.33439583e-02 2.04174861e-01]
 [2.82380670e-01 2.31594630e-02 2.37093680e-02 ... 3.01467091e-01
  3.02852720e-01 3.76921222e-02]
 [4.75906860e-03 7.25496793e-04 1.22725056e-03 ... 2.47282639e-01
  6.50194474e-03 5.77207562e-03]]
